# Kaggle竞赛的蘑菇数据集

Logstic回归
正则

## Importing all the libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

## Reading the file

还是蘑菇数据集，直接采用Kaggle竞赛中22维特征
https://www.kaggle.com/uciml/mushroom-classification

调用head函数看看每个特征的基本情况

In [2]:
# path to where the data lies
dpath = './data/'
data = pd.read_csv(dpath+"mushrooms.csv")
data.head(6)

FileNotFoundError: File b'./data/mushrooms.csv' does not exist

In [ ]:
data.info()

### Let us check if there is any null values

很幸运，该数据没有空值／缺失数据

In [ ]:
data.isnull().sum()

### 特征为字符串，需转换成数值型
LabelEncoder
也可以试试one hot编码

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for col in data.columns:
    data[col] = labelencoder.fit_transform(data[col])
 
data.head()

Separating features and label

In [ ]:
X = data.iloc[:,1:23]  # all rows, all the features and no labels
y = data.iloc[:, 0]  # all rows, label only

### Splitting the data into training and testing dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=4)

## default Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

#default, L2正则，C=1
model_LR= LogisticRegression()

In [ ]:
model_LR.fit(X_train,y_train)

In [ ]:
y_prob = model_LR.predict_proba(X_test)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_prob > 0.5, 1, 0) # This will threshold the probabilities to give class predictions.
model_LR.score(X_test, y_pred)

accuracy 

In [ ]:
auc_roc=metrics.roc_auc_score(y_test,y_pred)
auc_roc

## Logistic Regression(Tuned model)

# logistic回归的需要调整超参数有：C（正则系数，一般在log域（取log后的值）均匀设置调优）和正则函数penalty（L2/L1）
目标函数为：J(theata) = sum(logloss(f(xi), yi)) + 1/C * penalty
logistic回归: f(xi) = sigmoid(sum(wj * xj))
logloss为负log似然损失（请见课件）
L2 penalty：sum(wj^2)
L1 penalty: sum(abs(wj))

在sklearn框架下，不同学习器的参数调整步骤相同：
1. 设置候选参数集合
2. 调用GridSearchCV
3. 调用fit

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics

LR_model= LogisticRegression()

tuned_parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000,10000] ,
              'penalty':['l1','l2']
                   }

### CV

In [ ]:
from sklearn.model_selection import GridSearchCV

LR= GridSearchCV(LR_model, tuned_parameters,cv=10)

In [ ]:
LR.fit(X_train,y_train)

In [ ]:
print(LR.best_params_)

In [ ]:
y_prob = LR.predict_proba(X_test)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_prob > 0.5, 1, 0) # This will threshold the probabilities to give class predictions.
LR.score(X_test, y_pred)

In [ ]:
auc_roc=metrics.roc_auc_score(y_test,y_pred)
auc_roc

In [ ]:
LR_model

可以根据特征重要性进行特征选择

In [ ]:
from numpy import sort
from sklearn.feature_selection import SelectFromModel

# Fit model using each importance as a threshold
thresholds = sort(model_XGB.feature_importances_)
for thresh in thresholds:
  # select features using threshold
  selection = SelectFromModel(model_XGB, threshold=thresh, prefit=True)
  select_X_train = selection.transform(X_train)
  # train model
  selection_model = XGBClassifier()
  selection_model.fit(select_X_train, y_train)
# eval model
  select_X_test = selection.transform(X_test)
  y_pred = selection_model.predict(select_X_test)
  predictions = [round(value) for value in y_pred]
  accuracy = accuracy_score(y_test, predictions)
  print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1],
      accuracy*100.0))